In [ ]:
Пайплайн
========
Часть 2.
1. Авторизуется в Google Photos
2. Подключается к Google Drive, забирая все ссылки из указанной папки 
и сохраняет результат в google_drive_links.csv

In [ ]:
# Вот полный скрипт, который автоматически формирует google_drive_links.csv, подключаясь к Google Drive, 
# забирая все ссылки из указанной папки, и сохраняет результат в CSV-файл.

In [19]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build_from_document
import pandas as pd
import os
from tqdm import tqdm

# === Настройки ===
SCOPES = ['https://www.googleapis.com/auth/photoslibrary.readonly']
FOLDER_ID = '1em81MElkxnaue5r92e9hPGezgOuw4nRL'
ALBUM_NAMES = ["PhotoMap 2022-2025", "PhotoMap 2019-2021", "PhotoMap 2017-2019", "PhotoMap 2014-2016", "PhotoMap 2010-2013", "PhotoMap 08-09"]
OUTPUT_CSV = 'csv/google_drive_links.csv'

# === 🌐 Авторизация Google Photos и Google Drive
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

flow = InstalledAppFlow.from_client_secrets_file('client_secrets.json', SCOPES)
credentials = flow.run_local_server(port=0)

# === Загрузка Discovery документа
with open("photoslibrary_v1_discovery.json", "r") as f:
    discovery_doc = f.read()
service = build_from_document(discovery_doc, credentials=credentials)

# === 🔄 Получаем список файлов из Google Drive
print("🔍 Получаем список файлов из Google Drive...")
file_list = drive.ListFile({'q': f"'{FOLDER_ID}' in parents and trashed=false"}).GetList()

drive_data = {
    'filename': [file['title'] for file in file_list],
    'file_id': [file['id'] for file in file_list],
    'link': [f"https://drive.google.com/uc?export=view&id={file['id']}" for file in file_list],
    'description': ['No Description'] * len(file_list)  # УНИФИЦИРОВАЛИ!
}

df_links = pd.DataFrame(drive_data)
df_links.set_index('filename', inplace=True)

# === 🔍 Поиск нужных альбомов ===
print("🔍 Поиск альбомов в Google Photos...")
albums = []
nextPageToken = None

while True:
    response = service.albums().list(pageSize=50, pageToken=nextPageToken).execute()
    albums.extend(response.get('albums', []))
    nextPageToken = response.get('nextPageToken')
    if not nextPageToken:
        break

target_albums = {album['title']: album['id'] for album in albums if album['title'] in ALBUM_NAMES}

# === 💾 Обновляем DataFrame с данными из Google Photos
for album_name, album_id in target_albums.items():
    print(f"\n📂 Альбом: {album_name}")
    nextPageToken = None
    
    with tqdm(desc=f"Загрузка из {album_name}", unit="фото", leave=True) as pbar:
        while True:
            results = service.mediaItems().search(
                body={"albumId": album_id, "pageSize": 100, "pageToken": nextPageToken}
            ).execute()

            items = results.get('mediaItems', [])
            nextPageToken = results.get('nextPageToken')

            if not items:
                break

            for item in items:
                description = item.get('description', 'No Description')
                filename = item.get('filename', 'No Filename')
                
                if filename in df_links.index:
                    df_links.at[filename, 'description'] = description

            # === Динамическое обновление прогресса ===
            pbar.update(len(items))

            if not nextPageToken:
                break

# === 💾 Сохранение в CSV
os.makedirs('csv', exist_ok=True)
df_links.to_csv(OUTPUT_CSV)
print(f"\n✅ Ссылки и описания успешно сохранены в {OUTPUT_CSV}")

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=849244129200-hdfqohk1rs46hjekajgu7pa4jqrn9sqj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=849244129200-hdfqohk1rs46hjekajgu7pa4jqrn9sqj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60396%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fphotoslibrary.readonly&state=BRHOu9Ewty3yHxsAM1CKmphse8quQq&access_type=offline
🔍 Получаем список файлов из Google Drive...
🔍 Поиск альбомов в Google Photos...

📂 Альбом: PhotoMap 2022-2025


Загрузка из PhotoMap 2022-2025: 653фото [00:06, 104.60фото/s]



📂 Альбом: PhotoMap 2019-2021


Загрузка из PhotoMap 2019-2021: 784фото [00:08, 95.72фото/s] 



📂 Альбом: PhotoMap 2017-2019


...


📂 Альбом: PhotoMap 2010-2013


Загрузка из PhotoMap 2010-2013: 257фото [00:02, 87.71фото/s]



📂 Альбом: PhotoMap 08-09


Загрузка из PhotoMap 08-09: 109фото [00:02, 53.68фото/s] 


✅ Ссылки и описания успешно сохранены в csv/google_drive_links.csv
